In [7]:
import imutils
import numpy as np
from PIL import Image, ImageOps
from collections import deque
import os
import random
import csv

In [8]:
def extract_all_non_white_points(image_path, num_points=68):
    with Image.open(image_path) as img:
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img = np.array(img)

        img = imutils.resize(img, width=500)
        pixels = np.array(img)

    white_threshold = 245
    non_white_pixels = np.where(np.any(pixels < white_threshold, axis=-1))
    non_white_point_coordinates = list(zip(non_white_pixels[1], non_white_pixels[0]))

    if len(non_white_point_coordinates) < num_points:
        raise ValueError("Not enough non-white points to sample the desired number of points.")
    sampled_points = random.sample(non_white_point_coordinates, num_points)

    return sampled_points

In [9]:
folder_path = 'images'
csv_file_path = 'random_points.csv'
image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.png'))]
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # 寫入標頭
    header = ['File Name'] + [f'Point {i}' for i in range(1, 69)]
    writer.writerow(header)
    
    # 迭代每一個檔案，提取點，並寫入CSV
    random.seed(42)
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        points = extract_all_non_white_points(image_path)
        #print(points)
        # 將每個點的(x,y)座標轉換成"(x,y)"格式的字符串
        points_str = [f"({x},{y})" for x, y in points]
        # 寫入檔名和所有點的座標
        writer.writerow([image_file] + points_str)